<a href="https://colab.research.google.com/github/peterphoenix/School-Projects/blob/master/Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Peter Phoenix - 2201735413

Natural Language Processing - LA01

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = {
    'Data': ['This was a great movie with a good cast, all of them hitting on all cylinders.', 
             "Even if you're a huge Sandler fan, please don't bother with this extremely disappointing comedy!",
             'A movie of outstanding brilliance and a poignant and unusual love story',
             'I had the misfortune to watch this rubbish on Sky Cinema Max in a cold winter night',
             'I am at a distinct disadvantage here. I have not seen the first two movies in this series', 
             "This program is a lot of fun and the title song is so catchy I can't get it out of my head"],
    'Label': ['Positive', 'Negative', 'Positive', 'Negative', 'Negative', 'Positive']
}
data = pd.DataFrame(data, columns=["Data", "Label"])
print(data)

                                                Data     Label
0  This was a great movie with a good cast, all o...  Positive
1  Even if you're a huge Sandler fan, please don'...  Negative
2  A movie of outstanding brilliance and a poigna...  Positive
3  I had the misfortune to watch this rubbish on ...  Negative
4  I am at a distinct disadvantage here. I have n...  Negative
5  This program is a lot of fun and the title son...  Positive


# A. Data Pre-processing

**Case folding**

Case folding berfungsi untuk mengubah setiap huruf menjadi huruf kecil.

Hal ini dilakukan agar kata yang sama dianggap sebagai satu kesatuan. Jika hal ini tidak dilakukan, kata "Happy" dan "happy" akan dianggap menjadi 2 buah kata yang berbeda. Tahap ini juga memudahkan dan meningkatkan ketepatan pada langkah selanjutnya.

In [ ]:
# Case folding
data['Data'] = data['Data'].str.lower()

print(data)

                                                Data     Label
0  this was a great movie with a good cast, all o...  Positive
1  even if you're a huge sandler fan, please don'...  Negative
2  a movie of outstanding brilliance and a poigna...  Positive
3  i had the misfortune to watch this rubbish on ...  Negative
4  i am at a distinct disadvantage here. i have n...  Negative
5  this program is a lot of fun and the title son...  Positive


**Tokenization**

Tokenization berfungsi untuk memisahkan text menjadi kata-kata terpisah.

Kita juga dapat menghilangkan tanda baca dalam kata tersebut jika kita menggunakan tokenizer yang tepat.

Tanda baca dihilangkan karena tanda baca tidak mengarah terhadap kelas tertentu. Jika kita menggunakan word_tokenize biasa, maka kata "I am happy." akan dipecah menjadi "I", "am", "happy", ".". Namun, jika kita menggunakan RegexpTokenizer, kata "I am happy." akan dipecah menjadi "I", "am", "happy" saja tanpa tanda baca titik.

In [ ]:
# Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer

# Jika kita menggunakan word_tokenize biasa dari nltk, masih terdapat tanda baca, oleh karena itu kita menggunakan RegexpTokenizer
token = RegexpTokenizer(r'\w+')

data['Data'] = data.apply(lambda row: token.tokenize(row['Data']), axis=1)
 
print(data)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                                                Data     Label
0  [this, was, a, great, movie, with, a, good, ca...  Positive
1  [even, if, you, re, a, huge, sandler, fan, ple...  Negative
2  [a, movie, of, outstanding, brilliance, and, a...  Positive
3  [i, had, the, misfortune, to, watch, this, rub...  Negative
4  [i, am, at, a, distinct, disadvantage, here, i...  Negative
5  [this, program, is, a, lot, of, fun, and, the,...  Positive


**Stop-words removal**

Stop-words removal berfungsi untuk menghilangkan kata-kata umum yang tidak memiliki makna tertentu.

Hal ini dilakukan karena kata-kata yang tidak memiliki makna tertentu tersebut, seperti kata "the" akan merusak hasil model nantinya. Kata "the" tersebut tidak mengarah terhadap suatu kelas tertentu, oleh karena itu perlu dihilangkan agar tidak masuk kedalam perhitungan.

In [ ]:
# Stop-words removal
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 

data['Data'] = data['Data'].apply(lambda x: [item for item in x if item not in stop_words])

print(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                                                Data     Label
0     [great, movie, good, cast, hitting, cylinders]  Positive
1  [even, huge, sandler, fan, please, bother, ext...  Negative
2  [movie, outstanding, brilliance, poignant, unu...  Positive
3  [misfortune, watch, rubbish, sky, cinema, max,...  Negative
4  [distinct, disadvantage, seen, first, two, mov...  Negative
5  [program, lot, fun, title, song, catchy, get, ...  Positive


**Stemming**

Stemming berfungsi untuk mengubah suatu kata menjadi bentuk paling sederhana dari kata tersebut, seperti kata  “I”, “am”, “tired” akan diubah menjadi “I”, “am”, “tire”. Stemming menggunakan morpheme, yaitu satuan terkecil dari sebuah bahasa, oleh karena itu kata "happiness" jika menggunakan stemming akan diubah menjadi "happi", bukan "happy".

Hal ini dilakukan agar kata seperti happiness dan happy tadi dianggap menjadi sebuah kata yang memiliki arti yang sama. 


In [ ]:
# Stemming
from nltk.stem import LancasterStemmer

data['Data'] = data['Data'].apply(lambda x: [LancasterStemmer().stem(y) for y in x])

print(data)

                                                Data     Label
0               [gre, movy, good, cast, hit, cylind]  Positive
1  [ev, hug, sandl, fan, pleas, both, extrem, dis...  Negative
2    [movy, outstand, bril, poign, unus, lov, story]  Positive
3  [misfortun, watch, rub, sky, cinem, max, cold,...  Negative
4   [distinct, disadv, seen, first, two, movy, sery]  Negative
5  [program, lot, fun, titl, song, catchy, get, h...  Positive


# B. Word Embedding

In [ ]:
import gensim 
from gensim.models import Word2Vec 

In [ ]:
vec = Word2Vec(data['Data'], min_count = 1, window = 3, size = 10)

print(vec)
print(vec.wv.vectors.shape)
print(vec.wv.vocab.keys())

Word2Vec(vocab=44, size=10, alpha=0.025)
(44, 10)
dict_keys(['gre', 'movy', 'good', 'cast', 'hit', 'cylind', 'ev', 'hug', 'sandl', 'fan', 'pleas', 'both', 'extrem', 'disappoint', 'comedy', 'outstand', 'bril', 'poign', 'unus', 'lov', 'story', 'misfortun', 'watch', 'rub', 'sky', 'cinem', 'max', 'cold', 'wint', 'night', 'distinct', 'disadv', 'seen', 'first', 'two', 'sery', 'program', 'lot', 'fun', 'titl', 'song', 'catchy', 'get', 'head'])


Dasar pemikiran dari Word2Vec adalah dimana setiap word atau kata dapat diterjemahkan menjadi sebuah vector angka dimana setiap kata berbeda memiliki vector yang berbeda dan setiap kata yang sama memiliki vector yang sama. Tujuan dari diubahnya kata menjadi vector angka adalah agar kita dapat melakukan operasi matematika pada data tersebut.

Proses Word2Vec menggunakan proses neural network. Pada awalnya, kita menginisialisasi vector input dari setiap kata. Vector tersebut berupa angka dengan ukuran sesuai dengan size Word2Vec, dimana size diatas adalah 10. Kemudian kita menentukan target dari input tersebut berupa kata yang berada di sebelah kata input, sesuai dengan ukuran window sizenya. Window size yang digunakan diatas adalah 3. Jika kita menggunakan window size = 2, maka cara tersebut dinamakan CBoW atau Continuous Bag of Words, namun lebih dari itu dinamakan dengan skipgram.

Setelah menentukan input dan outputnya masing-masing (setiap input dapat memiliki lebih dari 1 output, sesuai dengan window sizenya dan sesuai kemunculan dari kata tersebut), proses neural network akan dijalankan hingga hidden layer yang didapatkan dapat memetakan input menjadi target yang sesuai.

Proses yang terjadi dapat dijabarkan sebagai berikut:


1.   Input berupa vector angka dari kata tertentu dan target berupa vector angka dari kata yang menjadi target input tersebut.
2.   Weight diinisialiasi secara random.
3. Proses selanjutnya dijalankan seperti proses neural network biasa, dimana dilakukan perkalian matriks dari layer input menuju hidden ditambah dengan bias (jika ada) kemudian digunakan fungsi aktivasi yang berupa fungsi softmax. 
4. Langkah tersebut diulangi kembali untuk layer hidden menuju output. 
5. Kemudian hitung error dari layer output dan hidden. Untuk layer output, error dihitung dengan cara selisih dari output yang didapatkan dengan output yang diinginkan. Untuk layer hidden, error dihitung dengan cara melakukan perkalian matriks antara error layer output dengan weight pada hidden layer. 
6. Setelah mendapatkan error setiap layer, weight diupdate dengan cara selisih dari weight lama dengan error yang telah dikalikan dengan learning rate.
7. Langkah 3 hingga 6 diulangi hingga output yang diinginkan sesuai.

Setelah menjadi sesuai, hidden layer tersebutlah yang menjadi output  yang kita inginkan dari proses BPNN ini.


#C. Review detection model using BPNN

Output yang digunakan yaitu sesusai dengan output pada soal.

In [ ]:
output = data['Label']
print(output)

0    Positive
1    Negative
2    Positive
3    Negative
4    Negative
5    Positive
Name: Label, dtype: object


Untuk input, pertama kita perlu mencari banyaknya jumlah kata maksimal dalam satu input.

In [ ]:
max = 0
for i in range(data.shape[0]):
  temp = len(data['Data'][i])
  if max < temp:
    max = temp

print(max)

9


Karna max kata dalam satu input ada 9, maka jumlah feature yang dibuat yaitu 9*10(ukuran vector Word2Vec) = 90 feature. Dan karena kita mempunyai 6 input berbeda, maka dibuat array berukuran 6x90

Input pada BPNN yang akan digunakan berisi feature setiap kata yang ada dalam input, yang berarti setiap input memiliki jumlah kata*10 dan sisanya diisi oleh nilai 0.

Contohnya jika 1 input memiliki 2 buah kata, maka kata pertama menempati feature 0-9 dan kata kedua menempati feature 10-19, kemudian sisanya yaitu feature 20-89 berisi nilai 0.

Untuk membentuk input seperti itu, kita dapat memanfaatkan fungsi for dengan rincian:

1.   for pertama untuk baris inputan.
2.   for kedua untuk kata ke-j pada inputan di baris tersebut.
3.   for ketiga untuk nilai vector dari kata tersebut.


In [ ]:
input = np.zeros((6, 90), dtype = 'float64')

for i in range(data.shape[0]):
  ct = 0
  for j in range(len(data['Data'][i])):
    for k in range(vec.wv.vectors.shape[1]) :
      input[i][ct] = vec[data['Data'][i][j]][k]
      ct+=1

print(input)
print(input.shape)

[[ 0.01872313  0.0495575  -0.03518298 -0.00467991 -0.04263043  0.01422503
  -0.03249187  0.02335061  0.02648079  0.04800149  0.02520237  0.01018078
  -0.03892053 -0.04625027  0.01383292 -0.04281176  0.02569087  0.0240184
  -0.01292509  0.04920362 -0.03439813  0.03221051  0.04698032 -0.01958213
   0.0395551  -0.0396603  -0.00830341  0.00916785  0.0292665   0.03528667
  -0.03697141  0.03452208 -0.0444804  -0.01940611 -0.00778832 -0.00177132
   0.0461552   0.01311467 -0.04455207  0.01568138  0.02631926 -0.01900967
  -0.01618009 -0.01314419 -0.0154218  -0.01189275  0.00747666 -0.01804278
   0.00950475  0.02393494  0.00112207  0.04929164  0.00013444 -0.00737058
  -0.01878775  0.04685513 -0.03331408  0.01648496 -0.0482969   0.03081307
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


**BPNN - Inisialisasi Data**

*   Sebelum melakukan proses training didefinisikan terlebih dahulu fungsi aktivasi berupa fungsi sigmoid seperti dibawah.
*   Masukkan input dan output dataset
*   Input data dinormalisasi menggunakan MinMaxScaler karena hasil yang didapatkan setelah dilakukan Word2Vec sangatlah kecil (dapat dilihat diatas bahwa input berisi nilai 0.0006..., 0.0011..., dan seterusnya).
*   Output data dinormalisasi menggunakan OneHotEncoder karena output masih berupa string, harus diubah menjadi numeric. Dapat dilihat dibawah bahwa class 1 adalah positive dan class 0 adalah negative
*   Inisialisasi weight dan bias secara random, serta inisialisasi learning rate (alpha). Disini kita menggunakan 4 buah hidden layer.
*   Bagi dataset untuk training dan testing menggunakan train_test_split
*   np.argmax yang digunakan pada target_train dan target_test hanya untuk saat menghitung akurasi, recall, dan presisi dikarenakan setelah dilakukan OneHotEncoder pada output, hasil yang didapatkan adalah array 6x2 yang berupa kemungkinan suatu input berapa di kelas output tertentu. Contohnya nilai [0.7 0.8] berarti input tersebut memiliki 70% kemungkinan untuk berada di kelas 0 dan 80% kemungkinan berada di kelas 1. np.argmax mengubah array tersebut kembali menjadi 6x1 dengan cara memilih nilai terbesar dan mengembalikan indexnya.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Sigmoid function
def nonlin(x,deriv=False):
  if(deriv==True):
    return x*(1-x)
  return 1/(1+np.exp(-x))

# Input dataset
data = input

# Output dataset
target = output.values.reshape((len(output), -1))
print("Original Target:")
print(target)
data = MinMaxScaler().fit_transform(data)
target = OneHotEncoder(sparse = False).fit_transform(target)
print("Target After One Hot Encoding:")
print(target)
print("Output yang dimaksud setelah One Hot Encoding")
print(np.argmax(target, axis = 1))

np.random.seed(1)
alpha = 0.001

# Inisialisasi weight dan bias
syn0 = np.random.uniform(size=(90,4))
bias0 =np.random.uniform(size=(1,4))
syn1 = np.random.uniform(size=(4,2))
bias1 = np.random.uniform(size=(1,2))

X, feature_test, y, target_test = train_test_split(data, target, test_size=0.3)

target_train = np.argmax(y, axis = 1)
target_test = np.argmax(target_test, axis = 1)

Original Target:
[['Positive']
 ['Negative']
 ['Positive']
 ['Negative']
 ['Negative']
 ['Positive']]
Target After One Hot Encoding:
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
Output yang dimaksud setelah One Hot Encoding
[1 0 1 0 0 1]


**BPNN - Training Data**

*   Training dilakukan dengan epoch sebesar 5000.
*   Tipe training yang digunakan adalah batch learning, dimana setiap epoch menggunakan seluruh input sekaligus.
*   Forward propagation dilakukan dengan cara mengalikan matrix yang ada dari input ke hidden ditambahkan dengan bias kemudian dilakukan fungsi sigmoid, setelah itu langkah tersebut diulangi untuk hidden ke output.
*   Pada bagian backpropagation, kita menghitung error dan delta dimana error adalah selisih dari output yang didapatkan dengan target. Error pada l2 didapatkan dengan mengurangi l2 dengan output target, namun error pada l1 didapatkan dengan cara mengalikan error pada l2 dengan weight l1. Delta adalah error yang dikalikan dengan error itu sendiri yang sudah digunakan fungsi aktivasi (dalam kode ini berupa fungsi sigmoid).
*   Update weight dan bias 
*   3 langkah diatas diulangi sebanyak 5000 kali (sebanyak jumlah epoch).
*   Setelah selesai, hasil yang kita inginkan adalah weight pada l1 (layer 1 atau hidden layer) yang kemudian akan dilakukan forward propagation sekali lagi pada data yang ingin didapatkan hasilnya.


In [ ]:
for idx in range(5000):
  # Forward Propagation --> Batch Learning
  l1 = nonlin(np.dot(X,syn0) + bias0)
  l2 = nonlin(np.dot(l1,syn1) + bias1)

  # Backpropagation
  l2_error = y - l2
  l2_delta = l2_error * nonlin(l2, deriv = True)
  l1_error = l2_delta.dot(syn1.T)
  l1_delta = l1_error * nonlin(l1, deriv = True)

  # Update Weight dan Bias
  syn1 += l1.T.dot(l2_delta) * alpha
  bias1 += np.sum(l2_delta,axis=0,keepdims=True) * alpha
  syn0 += X.T.dot(l1_delta) * alpha
  bias0 += np.sum(l1_delta,axis=0,keepdims=True) * alpha

**BPNN - Output**

*   Untuk mendapatkan output yang diinginkan, lakukan forward propagation sekali lagi pada data yang ingin didapatkan outputnya.
*   Output yang didapatkan pada l2 masih berupa float, oleh karena itu perlu digunakan argmax untuk mendapatkan hasilnya.
*   Semua output terklasifikasi menjadi kelas 1, hal ini dapat dikarenakan data yang digunakan sangatlah sedikit.

In [ ]:
print("Weight:")
print(syn0)

l1 = nonlin(np.dot(feature_test,syn0) + bias0)
l2 = nonlin(np.dot(l1,syn1) + bias1)

result = np.argmax(l2, axis = 1)
print("Test result: ", result)

l1_train = nonlin(np.dot(X,syn0) + bias0)
l2_train = nonlin(np.dot(l1_train,syn1) + bias1)

result_train = np.argmax(l2_train, axis = 1)
print("Train result: ", result_train)

Weight:
[[4.17022004e-01 7.20324493e-01 1.14374480e-04 3.02332573e-01]
 [1.46755891e-01 9.23385947e-02 1.86260211e-01 3.45560727e-01]
 [3.96767474e-01 5.38816734e-01 4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 8.78117436e-01 2.73875931e-02 6.70467510e-01]
 [4.17304802e-01 5.58689828e-01 1.40386939e-01 1.98101489e-01]
 [8.00744568e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606663e-01 8.50442112e-02 3.90547832e-02]
 [1.69830420e-01 8.78142503e-01 9.83468338e-02 4.21107625e-01]
 [9.57889530e-01 5.33165285e-01 6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01 1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01]
 [1.03226006e-01 4.47893526e-01 9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01 1.93669579e-02 6.78835533e-01]
 [2.11628116e-01 2.65546659e-01 4.91573159e-01 5.33625451e-02]
 [5.74117605e-01 1.46728575e-01 5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 

#D. Performance evaluation

In [ ]:
from sklearn import metrics

confusion = metrics.confusion_matrix(target_test, result)
print("Confusion Matrix Test Dataset:")
print(confusion)

accuracy = metrics.accuracy_score(target_test, result)
print ("Accuracy Test Dataset :{}%".format(accuracy*100))

precision = metrics.precision_score(target_test, result, zero_division=1)
print("Precision Test Dataset :{}%".format(precision*100))

recall = metrics.recall_score(target_test, result, zero_division=1)
print("Recall Test Dataset :{}%".format(recall*100))



confusion = metrics.confusion_matrix(target_train, result_train)
print("\n\nConfusion Matrix Train Dataset:")
print(confusion)

accuracy = metrics.accuracy_score(target_train, result_train)
print ("Accuracy Train Dataset :{}%".format(accuracy*100))

precision = metrics.precision_score(target_train, result_train, zero_division=1)
print("Precision Train Dataset :{}%".format(precision*100))

recall = metrics.recall_score(target_train, result_train, zero_division=1)
print("Recall Train Dataset :{}%".format(recall*100))

Confusion Matrix Test Dataset:
[[0 2]
 [0 0]]
Accuracy Test Dataset :0.0%
Precision Test Dataset :0.0%
Recall Test Dataset :100.0%


Confusion Matrix Train Dataset:
[[0 1]
 [0 3]]
Accuracy Train Dataset :75.0%
Precision Train Dataset :75.0%
Recall Train Dataset :100.0%


Hasil yang didapatkan adalah accuracy 0%, precision 0%, dan recall 100% untuk test dataset yang hanya memiliki 2 buah data, dan accuracy 75%, precision 75%, dan recall 100% untuk train dataset yang memiliki 4 buah data.

Hasil ini sangatlah tidak baik, dikarenakan jumlah dataset yang sangat sedikit baik untuk training maupun untuk testing, oleh karena itu proses traning tidak berjalan dengan baik, dan data akan lebih cenderung mengarah pada target yang memiliki output lebih banyak saat training (berdasarkan data diatas, dapat dilihat pada train dataset memiliki 3 buah data di kelas 1(positive) dan 1 buah data dengan kelas 0(negative), namun pada test dataset kedua data adalah kelas 0(negative)).

Hal lain yang mempengaruhi hasilnya adalah dimana setiap dapat memiliki input kata yang berbeda, oleh karena itu kata yang telah di training tidak digunakan saat testing, begitu juga sebaliknya kata yang muncul saat testing belum dilakukan training.